In [2]:
from kafka import KafkaConsumer, TopicPartition
from json import loads, JSONDecodeError
import requests
import json
import sys

# Kafka Consumer Configuration
KAFKA_BROKER = '10.76.92.36:9092'
KAFKA_TOPIC = 'amikum'
KAFKA_GROUP_ID = 'my-group'

# Webex API Configuration
WEBEX_API_URL = "https://webexapis.com/v1/messages"
WEBEX_ACCESS_TOKEN = 'Bearer NmRhNzBjZDMtYjNkZS00ZGViLTgxZjktOWNkYTE1YmZmNWIyZWE2MzRjY2QtMjhk_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f'  # Replace with your Webex API token
WEBEX_ROOM_ID = 'Y2lzY29zcGFyazovL3VzL1JPT00vOWQ4ZmQwMzAtZDMzNS0xMWVmLTgwOTItNWZhNzE1ZTQyMjZi'  # Replace with your Webex Room ID

# Webex Headers
headers = {
    'Authorization': WEBEX_ACCESS_TOKEN,
    'Content-Type': 'application/json'
}

# Initialize Kafka Consumer
consumer = KafkaConsumer(
    bootstrap_servers=[KAFKA_BROKER],
    auto_offset_reset='earliest',  # Start reading from the beginning
    enable_auto_commit=True,  # Automatically commit offsets
    group_id=KAFKA_GROUP_ID,
    value_deserializer=lambda x: x.decode('utf-8', errors='ignore') if x else None,
    consumer_timeout_ms=10000  # Exit if no messages are received within 10 seconds
)

print("Kafka Consumer initialized. Waiting for messages...")

try:
    # Manually assign partitions
    partitions = consumer.partitions_for_topic(KAFKA_TOPIC)
    if partitions:
        print("Partitions available:", partitions)
        topic_partitions = [TopicPartition(KAFKA_TOPIC, p) for p in partitions]
        consumer.assign(topic_partitions)
        consumer.seek_to_beginning()  # Start from the beginning of the topic
    else:
        print("No partitions found for the topic. Exiting...")
        consumer.close()
        sys.exit()

    # Consume messages from Kafka and post to Webex
    for msg in consumer:
        # Check for empty messages
        if not msg.value:
            print("Received an empty message.")
            continue

        try:
            # Attempt to parse the message as JSON
            message = loads(msg.value)
            print(f"Received JSON message from partition {msg.partition}, offset {msg.offset}:")
            print(json.dumps(message, indent=4))  # Pretty print the message

            # Prepare Webex message payload (pretty-printed JSON message)
            payload = {
                "roomId": WEBEX_ROOM_ID,
                "text": "Anomaly Alert Received:\n```\n" + json.dumps(message, indent=4) + "\n```"
            }

            # Post message to Webex
            response = requests.post(WEBEX_API_URL, headers=headers, json=payload)

            # Check for success
            if response.status_code == 200:
                print(f"Message posted successfully to Webex.")
            else:
                print(f"Failed to post to Webex. Status: {response.status_code}, Response: {response.text}")

        except JSONDecodeError as e:
            print(f"Failed to parse message from partition {msg.partition}, offset {msg.offset}: {msg.value}. Error: {e}")
        except KeyError as e:
            print(f"Missing key in message: {e}")

except Exception as e:
    print(f"Error while consuming messages: {e}")
finally:
    # Ensure the consumer is closed properly
    consumer.close()
    print("Consumer closed.")


Kafka Consumer initialized. Waiting for messages...
Partitions available: {0}
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received an empty message.
Received JSON message from partition 0, offset 12:
{
    "anomalyId": "282160",
    "category": "CONFIGURATION",
    "startTs": "2025-01-10T08:51:29.938Z",
    "endTs": "2025-01-10T08:51:29.938Z",
    "activeTs": "2025-01-10T08:51:29.938Z",
    "fabricName": "VXLAN-1",
    "nodeNames": [
        ""
    ],
    "entityName": "Fabric",
    "resourceType": "Fabric_Configuration",
    "resourceName": "Fabric_Configuration",
    "description": "vrf lite subnet ip range duplicate with fabric: cat-vxlan",
    "severity": 2,
    "streamingAnomaly": false,
    "anomalyType": "above_threshold",
    "ac